<a href="https://colab.research.google.com/github/RiteshVaidya/mapupassessment/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
import pandas as pd

def calculate_distance_matrix(file_path: str) -> pd.DataFrame:
    """
    Calculate a distance matrix based on the 'dataset-3.csv'.

    Args:
        file_path (str): Path to the 'dataset-3.csv' file.

    Returns:
        pd.DataFrame: Distance matrix
    """
    # Read the 'dataset-3.csv' file
    df = pd.read_csv(file_path)

    # Initialize a matrix with zeros
    num_ids = max(df['id_end'].max(), df['id_start'].max())
    distance_matrix = pd.DataFrame(0, index=range(1, num_ids + 1), columns=range(1, num_ids + 1))

    # Fill the matrix with cumulative distances along known routes
    for _, row in df.iterrows():
        start, end, distance = row['id_start'], row['id_end'], row['distance']
        distance_matrix.at[start, end] = distance
        distance_matrix.at[end, start] = distance  # Accounting for bidirectional distances

    # Calculate cumulative distances
    for i in range(1, num_ids + 1):
        for j in range(1, num_ids + 1):
            for k in range(1, num_ids + 1):
                distance_matrix.at[i, k] += distance_matrix.at[i, j] + distance_matrix.at[j, k]

    # Set diagonal values to 0
    distance_matrix.values[[range(num_ids)]*2] = 0

    return distance_matrix

# Example usage:
result_distance_matrix = calculate_distance_matrix('dataset-3.csv')
print(result_distance_matrix)



#2
import pandas as pd

def unroll_distance_matrix(distance_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Unroll a distance matrix DataFrame to a DataFrame with columns 'id_start', 'id_end', and 'distance'.

    Args:
        distance_matrix (pd.DataFrame): Distance matrix DataFrame.

    Returns:
        pd.DataFrame: Unrolled DataFrame.
    """
    # Create a list to store unrolled data
    unrolled_data = []

    # Iterate through each pair of indices and corresponding distance
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            if id_start != id_end:
                distance = distance_matrix.at[id_start, id_end]
                unrolled_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

    # Create a DataFrame from the list
    unrolled_df = pd.DataFrame(unrolled_data)

    return unrolled_df

result_unrolled_df = unroll_distance_matrix(result_distance_matrix)
print(result_unrolled_df)




#3


import pandas as pd

def find_ids_within_ten_percentage_threshold(df: pd.DataFrame, reference_value: int) -> pd.Series:
    """
    Find all IDs from the 'id_start' column whose average distance lies within 10% of the reference value's average distance.

    Args:
        df (pandas.DataFrame): DataFrame containing distances with 'id_start' column.
        reference_value (int): Reference value from the 'id_start' column.

    Returns:
        pd.Series: Sorted Series of 'id_start' values within the specified percentage threshold.
    """
    # Filter DataFrame based on the reference value
    subset_df = df[df['id_start'] == reference_value]

    # Calculate the average distance for the reference value
    reference_avg_distance = subset_df['distance'].mean()

    # Calculate the lower and upper bounds for the threshold
    lower_threshold = reference_avg_distance * 0.9
    upper_threshold = reference_avg_distance * 1.1

    # Filter 'id_start' values within the specified percentage threshold
    result_series = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]['id_start']

    # Sort the result Series
    result_series = result_series.sort_values().reset_index(drop=True)

    return result_series


result_within_threshold = find_ids_within_ten_percentage_threshold(result_unrolled_df, reference_value)
print(result_within_threshold)


#4
import pandas as pd

def calculate_toll_rate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate toll rates for each vehicle type based on the unrolled DataFrame.

    Args:
        df (pandas.DataFrame): Unrolled DataFrame with information about toll routes and distances.

    Returns:
        pandas.DataFrame: DataFrame with toll rates for each vehicle type.
    """
    # Rate coefficients for each vehicle type
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        column_name = f'{vehicle_type}_toll_rate'
        df[column_name] = df['distance'] * rate_coefficient

    return df

result_with_toll_rates = calculate_toll_rate(result_unrolled_df)
print(result_with_toll_rates)


#5
import pandas as pd
import datetime

def calculate_time_based_toll_rates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate time-based toll rates for different time intervals within a day.

    Args:
        df (pandas.DataFrame): DataFrame with time-related information.

    Returns:
        pandas.DataFrame: DataFrame with time-based toll rates.
    """
    # Convert day columns to proper case strings
    df['start_day'] = df['start_day'].str.capitalize()
    df['end_day'] = df['end_day'].str.capitalize()

    # Function to calculate toll rates based on time interval
    def calculate_rate(row):
        # Define time intervals and corresponding discount factors
        weekday_intervals = [(datetime.time(0, 0, 0), datetime.time(10, 0, 0), 0.8),
                             (datetime.time(10, 0, 0), datetime.time(18, 0, 0), 1.2),
                             (datetime.time(18, 0, 0), datetime.time(23, 59, 59), 0.8)]
        weekend_interval = (datetime.time(0, 0, 0), datetime.time(23, 59, 59), 0.7)

        # Check if the day is a weekend or weekday
        if row['start_day'] in ['Saturday', 'Sunday']:
            interval = weekend_interval
        else:
            # Find the appropriate weekday interval based on start time
            for start, end, discount_factor in weekday_intervals:
                if start <= row['start_time'] <= end:
                    interval = (start, end, discount_factor)
                    break

        # Calculate discounted toll rate
        return row['distance'] * interval[2]


    df['time_based_toll_rate'] = df.apply(calculate_rate, axis=1)

    return df


result_with_time_based_toll_rates = calculate_time_based_toll_rates(result_with_time_columns)
print(result_with_time_based_toll_rates)
